In [32]:
%reload_ext autoreload
%autoreload 2

import openpyxl
import pandas as pd
import numpy as np

from synthasizer.table import Table, Cell
from synthasizer.transformations import *
from synthasizer.conditions import *
from synthasizer.heuristics import *
from synthasizer.error import ReconstructionError

In [64]:
sales = openpyxl.load_workbook("../data/icecream.xlsx")
sales = Table.from_openpyxl(sales['icecream']['A1':'G10'])
sales.color(0, 1)
sales.color(1, 1)
sales.color(2, 1)
sales.color(2, 0)
sales.color(6, 1)
display(sales)
display(sales.color_df)

           0        1     2     3       4      5       6
0       Type  Country  June  July  August  Total  Profit
1     Banana       BE   170   690     520   1380     YES
2                  DE   610   640     320   1570      NO
3                  DE   250   650     630   1530     YES
4  Chocolate       BE   560   320     140   1020     YES
5                  FR   430   350     300   1080     YES
6                                                       
7                  NL   210   280     270    760      NO
8  Speculaas       BE   300   270     290    860      NO
9    Vanilla       BE   610   190     670   1470     YES

,0,0,0,0,0,0,0
0,0,0,4,0,0,0,0
1,1,2,3,0,0,0,5
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0


In [22]:
sales_header = Header(1)(sales)
display(sales_header.df)
display(sales_header.color_df)

,Type,Country,June,July,August,Total,Profit
0,Banana,BE,170,690,520,1380,YES
1,,DE,610,640,320,1570,NO
2,,DE,250,650,630,1530,YES
3,Chocolate,BE,560,320,140,1020,YES
4,,FR,430,350,300,1080,YES
5,,,,,,,
6,,NL,210,280,270,760,NO
7,Speculaas,BE,300,270,290,860,NO
8,Vanilla,BE,610,190,670,1470,YES


,0,0,4,0,0,0,0
0,1,2,3,0,0,0,5
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0


In [13]:
AggregatedHeuristic(EmptyColumnHeuristic())(sales)

0.8428571428571429

In [12]:
AggregatedHeuristic(ValueColumnHeuristic())(sales)

0.8603174603174605

In [14]:
ColorRowHeuristic()(sales)

0.8333333333333334

In [23]:
WeightedHeuristic(
    [AggregatedHeuristic(EmptyColumnHeuristic()),
     AggregatedHeuristic(ValueColumnHeuristic()),
     ColorRowHeuristic()],
)(sales_header)

0.886243386243386

In [66]:
error = ReconstructionError(sales)

In [38]:
d2 = Delete(6, EmptyCondition())(
        Delete(4, StyleCondition('bold', True))(sales)
     )
d2

           0   1    2    3    4     5    6
0     Banana  BE  170  690  520  1380  YES
1             DE  610  640  320  1570   NO
2             DE  250  650  630  1530  YES
3  Chocolate  BE  560  320  140  1020  YES
4             FR  430  350  300  1080  YES
5             NL  210  280  270   760   NO
6  Speculaas  BE  300  270  290   860   NO
7    Vanilla  BE  610  190  670  1470  YES

In [42]:
error(d2)

1

In [65]:
final = Delete(6, EmptyCondition())(Fill(0)(Header(1)(sales)))
final

0       Type Country June July August Total Profit
0     Banana      BE  170  690    520  1380    YES
1                 DE  610  640    320  1570     NO
2                 DE  250  650    630  1530    YES
3  Chocolate      BE  560  320    140  1020    YES
4                 FR  430  350    300  1080    YES
5                 NL  210  280    270   760     NO
6  Speculaas      BE  300  270    290   860     NO
7    Vanilla      BE  610  190    670  1470    YES

In [67]:
final.color_df

,0,0,4,0,0,0,0
0,1,2,3,0,0,0,5
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0


In [77]:
folded = Fold(2, 4)(final)
folded.df

,Type,Profit,Total,Country,variable,value
0,Banana,YES,1380,BE,June,170
1,,NO,1570,DE,June,610
2,,YES,1530,DE,June,250
3,Chocolate,YES,1020,BE,June,560
4,,YES,1080,FR,June,430
5,,NO,760,NL,June,210
6,Speculaas,NO,860,BE,June,300
7,Vanilla,YES,1470,BE,June,610
8,Banana,YES,1380,BE,July,690
9,,NO,1570,DE,July,640


In [80]:
error(folded)

Counter()


0.0

In [78]:
folded.header

True